In [12]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [13]:
# read the ticker file
tickers = pd.read_csv("Tickers.csv",header=None)
# rename the column as "ticker"
tickers.columns=['ticker']
# create a list that contains all the tickers
ticker_lst = tickers['ticker'].tolist()

len(ticker_lst)

62

In [18]:
# define function call_tickers that takes a list of tickers and return a list of called tickers
def call_tickers(lst):
    # create an empty list to store the called tickers
    called_ticker_lst = []
    # write a for loop to iterate the ticker list
    for i in lst:
        # call each ticker and append it to the called ticker list
        called_ticker_lst.append(yf.Ticker(i))
    # return the called ticker list when finish
    return called_ticker_lst

# defein function filter_USD that takes a list of called tickers and return a list of only US listed tickers
def filter_USD(lst):
    # create a new list to store the us tickers
    usd_lst = []
    # write a for loop that iterates the called ticker list
    for i in lst:
        # use try and except to ignore the delisted tickers
        try:
            # if the currency of the stock is "USD"
            if i.info['currency'] == 'USD':
                # append the ticker to the usd list
                usd_lst.append(i)
        except:
            pass
    # return the usd list
    return usd_lst

# define function filter_daily_volumn that takes a called ticker list, start date, end date, 
# and returns a list of tickers with an average daily_volume of at least 10000 shares in that time period
def filter_daily_volume(lst,start,end):
    # write a for loop to iterate the called ticker list
    for i in lst:
        # extract the volume of the stock in the given time period and see if the avergae is less than 10000
        if i.history(start = start, end = end,interval='1d')['Volume'].mean() < 10000:
            # if yes, remove the ticker from the list
            lst.remove(i)
        # return the list with unqualified tickers removed
        return lst



In [17]:
called_tickers = call_tickers(ticker_lst)
usd_tickers = filter_USD_ver2(called)
good_tickers = filter_daily_volume(usd, '2021-07-02','2021-10-22')

In [ ]:
def prices(lst,start,end):
    prices = pd.DataFrame()
    for i in lst:
        prices[str(i)[24:-1]] = i.history(start = start,end=end,interval = '1d')['Close']
    return prices

In [ ]:
prices = prices(good_tickers,'2018-01-01','2021-10-25')

In [84]:
std = []
names=[]

for i in good_tickers:
    std.append(prices[str(i)[24:-1]].std())
    names.append(str(i)[24:-1])
    
std_df = pd.DataFrame(std)
std_df['tickers'] = pd.DataFrame(names)
std_df.set_index(std_df['tickers'],inplace=True)
std_df.drop('tickers',axis=1,inplace=True)
std_df.columns=['std']
#std_df.sort_values(by='std')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.